# Task 1

In [89]:
! pip install lxml
import pandas as pd
import numpy as np
np.random.seed(42)

1st read the Table. Actually the page contains many tables, but the required one is the first, therefore the [0] after the function call.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Now the columns have to be renamed according to the task.

In [3]:
df.rename(columns={"Postcode": "PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)

Next drop all rows where Borough is not assigned.

In [4]:
df = df[df.Borough != 'Not assigned']

So now the index is a bit messed up. But this can easily be changed:

In [5]:
df = df.reset_index(drop=True)

This next line is a bit more tricky. First we group the data by PostalCode and Borough, but we do not want to set them as index. We then Join the Neighborhoods into one string. Then we reset the index again, because it got messed up.
Last but not least, the joined column is renamed back to the name "Neighborhood".

In [6]:
df = pd.DataFrame(df.groupby(['PostalCode','Borough'], as_index=False)["Neighborhood"].apply(lambda x: ', '.join(x))).reset_index().rename(columns={0:"Neighborhood"})

If there are any rows, where the Neighborhood is not assigned, we just use the value from Borough:

In [7]:
df["Neighborhood"][df.Neighborhood == 'Not assigned'] = df.Borough[df.Neighborhood == 'Not assigned'] 

In [8]:
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [9]:
df.shape

(103, 3)

# Task 2

The geocoder tool did not work, so I had to use the csv file.

In [10]:
coord_df = pd.read_csv("https://cocl.us/Geospatial_data")

In [11]:
coord_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

The only thing that has to be done for this task is to merge both datasets on the PostalCode key

In [12]:
df = pd.merge(df, coord_df, on="PostalCode")

In [13]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


# Task 3

I decided on finding areas in Toronto, where it is most lively. Therefore I will end up clustering the dataset in 3 parts: calm, medium lively and party area.

In [106]:
! pip install folium
import folium
import os
import requests
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# get secret values from environment variables:
CLIENT_ID = os.environ.get('FS_ID')
CLIENT_SECRET = os.environ.get('FS_SECRET')
VERSION = '20191204'
LIMIT = 100

In [107]:
latitude = 43.8
longitude = -79.55

First lets have a look at what we have got.

In [16]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now get all the information from Foursquare that are needed later on:

In [18]:
if not 'toronto_venues' in locals(): # prevents this cell from running when the variable toronto_venues already exists
    toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                       latitudes=df['Latitude'],
                                       longitudes=df['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [19]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
...,...,...,...,...,...,...,...
2236,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,McDonald's,43.741757,-79.584230,Fast Food Restaurant
2237,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,Rogers Plus,43.741312,-79.585263,Video Store
2238,Northwest,43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
2239,Northwest,43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot['lat'] = toronto_venues["Neighborhood Latitude"]
toronto_onehot['lon'] = toronto_venues["Neighborhood Longitude"]

toronto_onehot

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,lat,lon
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.806686,-79.194353
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.806686,-79.194353
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.784535,-79.160497
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.784535,-79.160497
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.784535,-79.160497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.739416,-79.588437
2237,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,43.739416,-79.588437
2238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.706748,-79.594054
2239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,43.706748,-79.594054


Look at all the criterias we could use:

In [97]:
for c in toronto_onehot.columns:
    print(c)

Accessories Store
Afghan Restaurant
Airport
Airport Food Court
Airport Gate
Airport Lounge
Airport Service
Airport Terminal
American Restaurant
Antique Shop
Aquarium
Argentinian Restaurant
Art Gallery
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
Auto Garage
Auto Workshop
BBQ Joint
Baby Store
Bagel Shop
Bakery
Bank
Bar
Baseball Field
Baseball Stadium
Basketball Stadium
Beach
Bed & Breakfast
Beer Bar
Beer Store
Belgian Restaurant
Bike Shop
Bistro
Board Shop
Boat or Ferry
Bookstore
Boutique
Brazilian Restaurant
Breakfast Spot
Brewery
Bridal Shop
Bubble Tea Shop
Building
Burger Joint
Burrito Place
Bus Line
Bus Station
Business Service
Butcher
Cafeteria
Café
Cajun / Creole Restaurant
Camera Store
Candy Store
Caribbean Restaurant
Carpet Store
Cheese Shop
Chinese Restaurant
Chocolate Shop
Church
Climbing Gym
Clothing Store
Cocktail Bar
Coffee Shop
College Arts Building
College Auditorium
College Gym
College Rec Center
College Stadium
Colombian Restaurant
Comfort Food Restaurant
Com

In [99]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

Cluster the Neighborhoods by livelyhood. The number of Cafés, Bars, Pubs, is a good indicator if a neighborhood is a busy or a quiet area. 

In [100]:
criteria = ["Café", "Bar", "Pub", "Restaurant", "Beer Bar", "Wine Bar", "Strip Club", "Nightclub"]

In [101]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)[criteria]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [102]:
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [103]:
toronto_grouped[toronto_grouped["Cluster Labels"] == 2][criteria]

,Café,Bar,Pub,Restaurant,Beer Bar,Wine Bar,Strip Club,Nightclub
53,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
70,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
toronto_grouped[toronto_grouped["Cluster Labels"] == 1][criteria]

,Café,Bar,Pub,Restaurant,Beer Bar,Wine Bar,Strip Club,Nightclub
6,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
10,0.142857,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
21,0.176471,0.000000,0.000000,0.058824,0.000000,0.0,0.0,0.058824
33,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
49,0.157895,0.052632,0.026316,0.052632,0.026316,0.0,0.0,0.026316
85,0.150000,0.000000,0.050000,0.000000,0.000000,0.0,0.0,0.000000


### So we can see, that the cluster 1 is the most lively and cluster 0 the most calm. cluster 2 is in between.

In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.Accent_r(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped['lat'], toronto_grouped['lon'], toronto_grouped['Neighborhood'], toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### The Colors in the map mean have the following meaning:
- green: calm
- grey: medium lively
- yellow: party areas